# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,4.27,90,100,14.67,TF,1727391750
1,1,tura,25.5198,90.2201,22.11,98,100,1.35,IN,1727391751
2,2,badger,64.8000,-147.5333,6.66,85,100,2.06,US,1727391753
3,3,pacific grove,36.6177,-121.9166,19.12,92,3,3.13,US,1727391688
4,4,port elizabeth,-33.9180,25.5701,15.16,85,0,3.09,ZA,1727391755


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    color = "City",
    tiles = "OSM",
    frame_width = 500,
    frame_height= 500,
)
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cond_df = city_data_df.loc[((city_data_df["Max Temp"] <= 30.0) & (city_data_df["Max Temp"] >= 20.0))\
                                 & (city_data_df["Cloudiness"] == 0),:]

# Drop any rows with null values
ideal_cond_df = ideal_cond_df.dropna(how="any")

# Display sample data
ideal_cond_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
39,39,ha'il,27.5219,41.6907,27.01,26,0,1.03,SA,1727391640
71,71,hotan,37.0997,79.9269,21.72,23,0,1.08,CN,1727391639
109,109,prado,-17.3411,-39.2208,24.22,74,0,5.71,BR,1727391874
135,135,sao joao da barra,-21.6403,-41.0511,24.05,81,0,14.20,BR,1727391905
142,142,guerrero negro,27.9769,-114.0611,21.98,71,0,9.76,MX,1727391912


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cond_df[["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

C:\Users\mahna\AppData\Local\Temp\ipykernel_4620\1246398311.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
39,ha'il,SA,27.5219,41.6907,26,
71,hotan,CN,37.0997,79.9269,23,
109,prado,BR,-17.3411,-39.2208,74,
135,sao joao da barra,BR,-21.6403,-41.0511,81,
142,guerrero negro,MX,27.9769,-114.0611,71,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "limit":10,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Remove rows with "No hotel found"
hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != "No hotel found",:]

# Display sample data
hotel_df

Starting hotel search
ha'il - nearest hotel: فندق كلاسيك
hotan - nearest hotel: Hua Yi International Hotel (accepts foreigners)
prado - nearest hotel: Pousada Recanto do Prado
sao joao da barra - nearest hotel: Pousada Mediterrâneo
guerrero negro - nearest hotel: Plaza sal paraiso
rupert - nearest hotel: No hotel found
lewistown - nearest hotel: No hotel found
antonio enes - nearest hotel: Hotel Quirimbas
brookings - nearest hotel: Quality Inn
willmar - nearest hotel: Holiday Inn Express
antsiranana - nearest hotel: Central
arraial do cabo - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
masuguru - nearest hotel: No hotel found
remire-montjoly - nearest hotel: Complexe Belova
fomboni - nearest hotel: Mohéli La Plage
pangai - nearest hotel: Billy's Place
santa ana - nearest hotel: Holiday Inn Express
marshalltown - nearest hotel: Tremont on Main
interlaken - nearest hotel: Comfort Inn
copiapo - nearest hotel: Hotel Diego de Almeida
mersa matruh - nearest hotel: صقور

,City,Country,Lat,Lng,Humidity,Hotel Name
39,ha'il,SA,27.5219,41.6907,26,فندق كلاسيك
71,hotan,CN,37.0997,79.9269,23,Hua Yi International Hotel (accepts foreigners)
109,prado,BR,-17.3411,-39.2208,74,Pousada Recanto do Prado
135,sao joao da barra,BR,-21.6403,-41.0511,81,Pousada Mediterrâneo
142,guerrero negro,MX,27.9769,-114.0611,71,Plaza sal paraiso
176,antonio enes,MZ,-16.2325,39.9086,89,Hotel Quirimbas
193,brookings,US,44.3114,-96.7984,39,Quality Inn
197,willmar,US,45.1219,-95.0433,32,Holiday Inn Express
202,antsiranana,MG,-12.2787,49.2917,73,Central
235,remire-montjoly,GF,4.9167,-52.2667,83,Complexe Belova


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=650,
    frame_height=500,
    size="Humidity",
    color="City",
    hover_cols=["Hotel Name","Country"]
    )

# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)